# Tests for HanTa German

Some simple tests to check whether everything is doing what it is suposed to do.

In [1]:
import sys
sys.path.insert(1, '..')
import HanoverTagger as ht #Do not import form the package but from the parent folder where the latest source file is found

tagger = ht.HanoverTagger(r'../morphmodel_ger.pgz')

## A number of arbitrary examples

In [2]:
print(tagger.analyze('Überlebende'))
print(tagger.analyze('überlebende'))
print(tagger.analyze('Überlebende',taglevel=3))
print(tagger.analyze('überlebende',taglevel=3))

('Überlebende', 'NNA')
('überlebend', 'ADJ(A)')
('überlebend', [('überleb', 'VVnp'), ('end', 'PRESPART'), ('e', 'SUF_ADJ')], 'NNA')
('überlebend', [('überleb', 'VVnp'), ('end', 'PRESPART'), ('e', 'SUF_ADJ')], 'ADJ(A)')


In [3]:
print(tagger.analyze('Wohnzimmerschränke',taglevel=3))
print(tagger.analyze('Holzfußboden',taglevel=3))
print(tagger.analyze('Schneeballsystem',taglevel=3))
print(tagger.analyze('Exportstandorts',taglevel=3))

('wohnzimmerschrank', [('wohnzimmer', 'NN'), ('schränk', 'NN_VAR'), ('e', 'SUF_NN')], 'NN')
('holzfußbode', [('holzfußbode', 'NN'), ('n', 'SUF_NN')], 'NN')
('schneeballsystem', [('schnee', 'NN'), ('ball', 'NN'), ('system', 'NN')], 'NN')
('exportstandort', [('export', 'NN'), ('stand', 'NN'), ('ort', 'NN'), ('s', 'SUF_NN')], 'NN')


In [4]:
print(tagger.tag_word('verdachten'))
print(tagger.tag_word('verfahren'))
print(tagger.tag_word('Verfahren'))

[('NN', -17.968088002381307)]
[('VV(INF)', -13.004997963605383), ('VV(PP)', -13.69776085689267), ('NN', -14.56877801341071)]
[('NN', -8.871950978144888), ('VV(INF)', -19.731531081236866), ('VV(PP)', -20.754912675522245)]


In [5]:
print(tagger.tag_word('verdachten',cutoff=10))

[('NN', -17.968088002381307), ('VV(FIN)', -25.27261332515106), ('VV(INF)', -26.750057963605382), ('ADJ(A)', -27.507934110461136)]


## Evaluate tagging and lemmatising on train data

This is ofcourse not a proper evaluation as we use the training data to evaluate the program. However, if we don't get good results here, something went wrong for sure. The numbers we get here might give a kind of upperbound for what we can ecpect from a real evaluation.

Evaluation of tagging and lemmatisation is not a trivial task. First, the evaluation data might have been tagged according to some othe tagging scheme. A lot of different decisions can be made about the tags and lemmata for many unclear cases. Moreover most data sets contain a large number of errors. Often these originate form the tagger/stemmer/lemmatiser that was used for the initial annotation and were overlooked in the manual correction phases.  

In [6]:
def load(lines):
    data = []
    sent = []
    lastsentnr = 1
    for line in lines:
        (sentnr,word,lemma,stem,tag,morphemes,stemsub) = line.split('\t')
        if sentnr != lastsentnr:
            if len(sent) > 0:
                data.append((sentnr,sent))
            sent = []
            lastsentnr = sentnr
        sent.append((word,lemma,tag))
    return data

In [7]:
import codecs

datafile = codecs.open(r"labeledmorph_ger.csv", "r","utf-8")

morphdata = []
for line in datafile:
    if not line.startswith('-1'):
        morphdata.append(line)
    

In [8]:
testdata = load(morphdata)

### Evaluate the POS Tagging

In [9]:
def tag_evaluate(sents):
    correct = 0
    nr = 0

    for snr,sent in sents:
        ws = [w for (w,l,c) in sent]
        cs = [c for (w,l,c) in sent]
        pred_cs = tagger.tag_sent(ws,taglevel = 0)
        for i in range(len(ws)): 
            nr += 1
            if cs[i] == pred_cs[i]:
                correct += 1
            #else:
            #    c = cs[i].split('(')[0]
            #    pred_c = pred_cs[i].split('(')[0]
            #    if c != pred_c:
            #        print(snr,'\t',' '.join(ws))
            #        print(ws[i],cs[i],pred_cs[i])
            #        print()
        if nr%50 == 0:
            print(correct/nr,end='\r')
    return correct/nr

In [10]:
tag_evaluate(testdata)

0.9831241823295361

### Evaluate lemmatisation

In [17]:
def lemma_evaluate(sents):
    correct = 0
    nr = 0

    for snr,sent in sents:
        ws = [w for (w,l,c) in sent]
        ls = [l for (w,l,c) in sent]
        pred_ls = [l for _,l,_ in tagger.tag_sent(ws,taglevel = 1)]
        for i in range(len(ws)): 
            nr += 1
            if ls[i] == pred_ls[i]:
                correct += 1
            #elif ls[i].lower() != pred_ls[i].lower():
            #    print(snr,'\t',' '.join(ws))
            #    print(ws[i],ls[i],pred_ls[i])
            #    print()
        if nr%50 == 0:
            print(correct/nr,end='\r')
    return correct/nr

In [18]:
lemma_evaluate(testdata)


0.9716030506848728